#1.Code Python để tạo đồ thị có hướng

In [9]:
!pip install mwxml
import bz2
import pickle
import re
import mwxml
import networkx as nx
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm


In [10]:
# Đường dẫn đến file xml
dump_path = r"simplewiki-20240220-pages-articles-multistream.xml.bz2"

# Các files ouput
graph_file = "graph_with_attributes.pkl"
features_file = "article_features.csv"

In [11]:
def create_directed_graph_and_extract_features(dump_path):
    # Tạo đồ thị có hướng với networkx
    graph = nx.DiGraph()

    # Lưu các articles(các bài viết của wiki)
    article_titles = set()

    # List đế lưu các đặc trưng(features) của các article
    features = []

    # NLTK stopwords
    stop_words = set(stopwords.words("english"))

    # Mở file xml bằng thư viện bz2
    with bz2.open(dump_path, "rt", encoding="utf-8") as dump_file:
        dump = mwxml.Dump.from_file(dump_file)

        # Đếm tổng số articles
        total_count = 0
        for page in dump:
            total_count += 1
            if page.namespace == 0 and not page.redirect:
        #Cho tên các articles vào set article_titles để kiểm tra
                title = page.title
                article_titles.add(title)
                       
        # Reset để có thể lướt qua các articles lại từ đầu
        dump_file.seek(0)
        dump = mwxml.Dump.from_file(dump_file)

        # Lần lượt đi qua các trang để lấy tên articles và đặc trưng 
        # Sử dụng tdqm tạo process bar để kiểm tra tiến độ
        for page in tqdm(dump, total=total_count, unit="articles", desc="Processing articles"):
            if page.namespace == 0 and not page.redirect:
                # Lấy tên các articles
                title = page.title

                # Đi qua các revision
                for revision in page:
                    if revision.text:
                        # Lấy text từ revision
                        tokens = word_tokenize(revision.text)

                        # Tạo bag-of_words
                        bag_of_words = " ".join(tokens)

                        # Lấy các đường links trong revision
                        hyperlinks = re.findall(r"\[\[(.*?)]]", revision.text)
                        hyperlinks_cleaned = [hyperlink.split("|")[0] for hyperlink in hyperlinks]

                        # Đếm số từ và link có trong articles
                        word_count = len(tokens)
                        link_count = len(hyperlinks_cleaned)

                        # Thêm node vào graph và các attributes cần để chạy heuristic function cho A*
                        attributes = {
                            "Word Count": word_count,
                            "Link Count": link_count,
                            "Bag of Words": bag_of_words
                        }
                    # Kiểm tra xem article có trong graph chưa để cho vào
                    if title in graph.nodes:
                        node_attributes = graph.nodes[title]
                        node_attributes.update(attributes)
                    else:
                    #Nếu có thì cập nhật attributes
                        graph.add_node(title, **attributes)

                    features.append((title, word_count, link_count, bag_of_words))
                    for hyperlink in hyperlinks_cleaned:
                        #Kiếm tra xem link có dẫn đến 1 article không
                        if hyperlink in article_titles:
                            if hyperlink not in graph.nodes:
                                # Nếu article chưa có thì tạo làm node
                                graph.add_node(hyperlink)
                            # Thêm cạnh vào graph    
                            graph.add_edge(title, hyperlink)

    # Lưa graph lại dưới file pickle
    with open(graph_file, "wb") as file:
        pickle.dump(graph, file)

    # Tạo DataFrame với các đặc trưng
    df = pd.DataFrame(features, columns=["Title", "Word Count", "Link Count", "Bag of Words"])

    # Lưu DataFrame với các đặc trưng vao CSV file
    df.to_csv(features_file, index=False)

    return graph, df


In [12]:
# Gọi hàm
graph, df = create_directed_graph_and_extract_features(dump_path)

Processing articles:   0%|          | 0/472626 [00:00<?, ?articles/s]

In [ ]:
# Thông báo khi hoàn thành
print('Finished!')